In [138]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from time import sleep as time_sleep
import pandas as pd
from bs4 import BeautifulSoup as BS

In [142]:
url = 'https://idk.ru/'
time_wait = 0.5 #sec

driver = uc.Chrome()
driver.get(url)



def _get_table(driver: uc.Chrome, class_wheat: str, name_wheat:str) -> pd.DataFrame:
    table = driver.find_element(
                            by=By.ID,
                            value='index-container'                        
                            ) \
                    .find_elements(
                            by=By.CSS_SELECTOR,
                            value='tr'
                        )

    spisok = []

    for elem in table:
        spisok.append(list(map(lambda x: x.text, elem.find_elements(by=By.CSS_SELECTOR, value='td'))))

    table_df = pd.DataFrame(spisok, columns=('Город', 'Цена', '????'))
    table_df[['Цена', '????']] = table_df[['Цена', '????']].astype(int)
    table_df.insert(0, 'Группа', class_wheat)
    table_df.insert(1, 'Наименование', name_wheat)
    return table_df
    

In [143]:
indexs = driver.find_element(
                        by=By.ID,
                        value='index-tabs-container'
                        ) \
                .find_elements(
                        by=By.CLASS_NAME,
                        value='main-section'
                    )

wheat_df = pd.DataFrame()

for elem in indexs:
    elem.click()
    
    time_sleep(time_wait)
    table_df = _get_table(
                    driver, 
                    elem.text, 
                    elem.get_attribute('title')
                    )
    wheat_df = pd.concat((wheat_df, table_df), axis=0, ignore_index=True)
    
wheat_df.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Группа        114 non-null    object
 1   Наименование  114 non-null    object
 2   Город         114 non-null    object
 3   Цена          114 non-null    int32 
 4   ????          114 non-null    int32 
dtypes: int32(2), object(3)
memory usage: 3.7+ KB


In [144]:
auction = driver.find_element(
                    by=By.CLASS_NAME,
                    value='links'
                ) \
                .find_element(
                    by=By.LINK_TEXT,
                    value='Торги'
                ) 

auction.click()
time_sleep(time_wait*4)


soup = BS(driver.page_source, 'html.parser')
table = soup.find('table', class_='grid').find('tbody').find_all('tr')


table_auction = []
for row in table:
    table_auction.append(list(row.stripped_strings))

auction_df = pd.DataFrame(
                    data=table_auction[1:],
                    columns=table_auction[0]
                )

auction_df['Объем'] = auction_df['Объем'].str.extract(r'(\d*)').astype(int)
auction_df[['№', 'Цена', 'Детали']] = auction_df[['№', 'Цена', 'Детали']].astype(int)

auction_df = auction_df.rename(columns={'Объем':'Объем, т'})
auction_df

,№,Дата,Операция,Товар,Организация,Объем,Цена,Торг,Поставка,Условия,Регион,Детали
0,104819,сегодня 09:13,продам,Ячмень Фуражн.,АО СЭК,25000 т,14500,-,Июль 2023,FOB,Астраханская обл.,0
1,104818,сегодня 09:07,продам,Кукуруза на кормовые цели,АО СЭК,3200 т,18500,-,Июль 2023,DAP,Азербайджан,0
2,104817,сегодня 09:01,продам,Пшеница Мягк. 4 кл.,АО СЭК,25000 т,13800,-,Июль 2023,FOB,Краснодарский край,0
3,104816,сегодня 08:59,продам,Пшеница Мягк. 3 кл.,АО СЭК,55000 т,15200,-,Июль 2023,FOB,Краснодарский край,0
4,104533,15.06.23 11:11,куплю,Пшеница Мягк. 4 кл.,ООО «Агропромцентр АВА»,20000 т,11500,-,Июнь 2023,CPT,Ростовская обл.,21
5,104532,15.06.23 11:10,куплю,Пшеница Мягк. 3 кл.,ООО «Агропромцентр АВА»,5000 т,11000,-,Июнь 2023,EXW,Воронежская обл.,0
6,104531,15.06.23 11:09,куплю,Пшеница Мягк. 4 кл.,ООО «Агропромцентр АВА»,15000 т,10000,-,Июнь 2023,EXW,Воронежская обл.,21
7,104530,15.06.23 11:06,куплю,Пшеница Мягк. 3 кл.,ООО «Агропромцентр АВА»,10000 т,11000,-,Июнь 2023,CPT,Украина,0


In [147]:
with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
    wheat_df.to_excel(writer, sheet_name='Индексы цен', index=0)
    auction_df.to_excel(writer, sheet_name='Торги', index=0)

In [148]:
driver.quit()